In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
import logging
from collections import namedtuple
import pickle
import multiprocessing
import time


logger = logging.getLogger()


In [ ]:
links_to_parse = []
base_page = "http://www.nuforc.org/webreports/"
resp = requests.get(url=base_page + "ndxshape.html")
parsed_data = []

In [ ]:
t = time.time()

def get_data_from_table(url:str,generate_desc:bool = False):
    soup = BeautifulSoup(requests.get(url).text,"html.parser")
    header = [i.text for i in  soup.thead.findAll("th")]
    data = []
    for row in soup.tbody.findAll("tr"):
        long_desc = ""
        if generate_desc:
            try:
                long_desc =  BeautifulSoup(requests.get(base_page+row.find("a").get("href")).text,"html.parser").tbody.findAll("td")[1].text
            except Exception as e:
                logger.log(level=logging.INFO,msg="could not retrieve long description")
        i = 0
        entry = {}
        for cell in row.findAll("td"):
            entry[header[i]] = cell.text
            i+=1
        entry["Description"] = long_desc
        data.append(entry)
    return data


# Entry = namedtuple("Entry",['Date / Time', 'City', 'State', 'Shape', 'Duration', 'Summary', 'Posted'])

bs = BeautifulSoup(resp.text,"html.parser")
for link in bs.table.tbody.findAll("a"):
    links_to_parse.append(base_page+link.get('href'))

    
#links_to_parse = ["http://www.nuforc.org/webreports/ndxschanged.html","http://www.nuforc.org/webreports/ndxsCrescent.html","http://www.nuforc.org/webreports/ndxsCross.html","http://www.nuforc.org/webreports/ndxsDelta.html","http://www.nuforc.org/webreports/ndxsDome.html","http://www.nuforc.org/webreports/ndxsFlare.html","http://www.nuforc.org/webreports/ndxsHexagon.html","http://www.nuforc.org/webreports/ndxspyramid.html","http://www.nuforc.org/webreports/ndxsRound.html"]

for link in links_to_parse:
    data = get_data_from_table(link,False)
    parsed_data.extend(data)
    print("done: "+link +" count: "+len(data))
    
print(time.time()-t)
print("checking count, should be around 113241 : "+len(parsed_data))

csv.register_dialect("semicolon",delimiter=";")
with open("data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    dw.writerows(parsed_data)


In [ ]:
with open("data.csv","w",newline="",encoding="utf-8") as f:
    dw = csv.DictWriter(f,parsed_data[0].keys(),dialect="semicolon")
    dw.writeheader()
    for item in parsed_data :
        try:
            dw.writerow(item)
        except Exception as e:
            print(item)

In [ ]:
with open("save.pckl","wb") as f:
    pickle.dump(parsed_data,f)
